In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import blob_dog
import torch

/home/lathika/miniconda3/envs/ges/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
a = torch.Tensor(np.random.rand(10,2))
mask = a[:,0]>0.5
print(a)
print(mask)
print(mask.squeeze())
print(a[mask])

tensor([[0.2776, 0.2274],
        [0.8167, 0.3409],
        [0.2464, 0.2059],
        [0.4103, 0.7414],
        [0.9473, 0.1951],
        [0.8138, 0.6273],
        [0.8388, 0.7032],
        [0.0275, 0.2398],
        [0.7290, 0.0729],
        [0.9471, 0.6953]])
tensor([False,  True, False, False,  True,  True,  True, False,  True,  True])
tensor([False,  True, False, False,  True,  True,  True, False,  True,  True])
tensor([[0.8167, 0.3409],
        [0.9473, 0.1951],
        [0.8138, 0.6273],
        [0.8388, 0.7032],
        [0.7290, 0.0729],
        [0.9471, 0.6953]])
